In [2]:
import pandas as pd
import numpy as np
import datetime as datetime
import os
os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [3]:
keys = ['8to9', '9to10', '10to11', '11to12','12to13','13to14','14to15','15to16','16to17','17to18','18to19','19to20']

In [14]:
# read data
episode_original_cloud_ids = [201, 203, 206, 210, 229] 
eod_original_cloud_ids = [201, 203, 206, 221, 229] 

puff_episode = pd.read_csv(dir + 'puff-episode.csv')
puff_episode = puff_episode.drop(['offset'], axis = 1)
eod_ema = pd.read_csv(dir + 'eod-ema.csv')
eod_ema = eod_ema.drop(['offset'], axis = 1)

puff_episode_alt = pd.read_csv('puff-episode-alternative.csv')
eod_ema_alternative = pd.read_csv(dir + 'eod-ema-alternative.csv')

puff_episode_backup = pd.read_csv('puff-episode-backup.csv')
eod_ema_backup = pd.read_csv(dir + 'eod-ema-backup.csv')

temp_puff_episode = puff_episode[puff_episode['participant_id'].isin(episode_original_cloud_ids)]
temp_puff_episode_alt = puff_episode_alt[~puff_episode_alt['participant_id'].isin(episode_original_cloud_ids)]

puff_complete = pd.concat([temp_puff_episode, temp_puff_episode_alt, puff_episode_backup])

temp_eod_original = eod_ema[eod_ema['participant_id'].isin(eod_original_cloud_ids)]
temp_eod_alt = eod_ema_alternative[~eod_ema_alternative['participant_id'].isin(eod_original_cloud_ids)]

eod_complete = pd.concat([temp_eod_original, temp_eod_alt, eod_ema_backup])

puff_complete.columns

Index([u'timestamp', u'event', u'participant_id', u'date', u'hour', u'minute',
       u'day_of_week'],
      dtype='object')

In [15]:
# Make a list of all contingent event-times between 8AM and 8PM
# Throw away observations for 'when_smoke' is nan or 
# 'More than 30 minutes' to ensure we can calculate a meaningful 
# quantity.
days_smoked = {}
for index, row in puff_complete.iterrows():
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    date = (time.year, time.month, time.day, time.hour)
    if row['participant_id'] not in days_smoked:
        days_smoked[row['participant_id']] = set()
    if 8 <= date[3] < 20:        
        days_smoked[row['participant_id']].add(date)

days_smoked

{201.0: {(2017, 6, 24, 17)},
 202.0: {(2017, 6, 27, 8)},
 203.0: set(),
 204.0: {(2017, 8, 14, 16),
  (2017, 8, 14, 17),
  (2017, 8, 14, 19),
  (2017, 8, 15, 9),
  (2017, 8, 15, 10),
  (2017, 8, 15, 12),
  (2017, 8, 15, 14)},
 205.0: {(2017, 8, 18, 16),
  (2017, 8, 18, 17),
  (2017, 8, 18, 19),
  (2017, 8, 19, 9),
  (2017, 8, 19, 10),
  (2017, 8, 20, 11),
  (2017, 8, 21, 11),
  (2017, 8, 22, 14),
  (2017, 8, 22, 15),
  (2017, 8, 22, 16),
  (2017, 8, 22, 17),
  (2017, 8, 23, 12),
  (2017, 8, 23, 13),
  (2017, 8, 23, 14),
  (2017, 8, 23, 15),
  (2017, 8, 23, 18),
  (2017, 8, 25, 9),
  (2017, 8, 27, 15),
  (2017, 8, 28, 9),
  (2017, 8, 28, 12),
  (2017, 8, 28, 13),
  (2017, 8, 28, 14),
  (2017, 8, 28, 17),
  (2017, 8, 30, 10),
  (2017, 8, 30, 18)},
 206.0: {(2017, 9, 25, 9)},
 207.0: {(2017, 9, 22, 15),
  (2017, 9, 23, 16),
  (2017, 9, 25, 11),
  (2017, 9, 25, 18),
  (2017, 9, 26, 13),
  (2017, 9, 27, 11),
  (2017, 9, 27, 12),
  (2017, 9, 27, 13),
  (2017, 9, 27, 14),
  (2017, 9, 27, 16),

In [16]:
# Construct a list of id + dates for eod
# Use to look 
eod_dates = []
for irow in range(0,eod_complete.shape[0]):
    row = eod_complete.iloc[irow]
    if row['status'] == "MISSED":
        continue
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    if time.hour  == 0 or time.hour == 1:
        date = np.array([row['participant_id'], time.year, time.month, time.day-1])
        date = np.append(date, np.array(row[keys]))
    else:
        date = np.array([row['participant_id'], time.year, time.month, time.day])
        date = np.append(date, np.array(row[keys]))
    eod_dates.append(date)
    
eod_dates = np.asarray(eod_dates)
eod_dates


array([[201, 2017, 6, ..., 0.0, 0.0, 0.0],
       [201, 2017, 6, ..., 0.0, 0.0, 0.0],
       [203, 2017, 8, ..., 0.0, 0.0, 0.0],
       ...,
       [236, 2018, 7, ..., 0.0, 0.0, 0.0],
       [237, 2018, 7, ..., 0.0, 0.0, 0.0],
       [237, 2018, 7, ..., 0.0, 0.0, 0.0]], dtype=object)

In [33]:
# For participants with both EC and EOD measurements,
# on days when you give both, we ask whether they agree,
# up to the current hour, or +- 1 hour in either direction.
# The +-1 is max/min by 8AM and 8PM respectively.
matching_counts = []
max_iloc = 15; min_iloc = 4
for id in set(days_smoked.keys()) & set(eod_dates[:,0]):
    eod_dates_id = np.where(eod_dates[:,0] == id) 
    eod_dates_subset = eod_dates[eod_dates_id[0],:]
    total_count_id = 0
    hour_count_id_true = 0
    twohour_count_id_true = 0
    for ec_time in days_smoked[id]:
        row_iloc = np.where((eod_dates_subset[:,1:4] == ec_time[0:3]).all(axis=1))[0]
        if not row_iloc.size > 0:
            continue
        total_count_id+=1
        row = eod_dates_subset[row_iloc][0]
        ec_iloc = range(8,20).index(ec_time[3])+4
        if row[ec_iloc]==1:
            hour_count_id_true+=1
        if any(row[range(max(min_iloc, ec_iloc-1), min(max_iloc, ec_iloc+1)+1)] == 1):
            twohour_count_id_true+=1
    matching_counts.append(np.array([total_count_id, hour_count_id_true, twohour_count_id_true], dtype='f'))

matching_counts = np.asarray(matching_counts)

matching_counts = np.delete(matching_counts, (np.where(matching_counts[:,0] == 0)[0]), axis=0)

print matching_counts

fraction_per_id_onehour = np.divide(matching_counts[:,1],matching_counts[:,0])
fraction_per_id_twohour = np.divide(matching_counts[:,2],matching_counts[:,0])

aggregate_matching_counts = np.sum(matching_counts, axis=0)

aggregate_frac_onehour = aggregate_matching_counts[1]/aggregate_matching_counts[0]
aggregate_frac_twohour = aggregate_matching_counts[2]/aggregate_matching_counts[0]

print 'Current hour only:'
print 'Aggregated data, Fraction agreement between EC and EOD: %s' % (np.round(aggregate_frac_onehour,3))
print 'Mean of Fraction agreement across indidivuals: %s' % (np.round(np.mean(fraction_per_id_onehour),3))
print 'Standard deviation of Fraction agreement across indidivuals: %s' %  (np.round(np.std(fraction_per_id_onehour),3))
print
print 'Plus-minus one hour:'
print 'Aggregated data, Fraction agreement between EC and EOD: %s' % (np.round(aggregate_frac_twohour,3))
print 'Mean of Fraction agreement across indidivuals: %s' % (np.round(np.mean(fraction_per_id_twohour),3))
print 'Standard deviation of Fraction agreement across indidivuals: %s' %  (np.round(np.std(fraction_per_id_twohour),3))



[[ 1.  0.  1.]
 [ 7.  7.  7.]
 [25.  3.  6.]
 [ 1.  0.  0.]
 [10.  0.  0.]
 [ 5.  2.  4.]
 [ 1.  1.  1.]
 [15.  4.  7.]
 [ 7.  3.  7.]
 [21. 16. 16.]
 [ 5.  1.  2.]
 [ 7.  1.  1.]
 [ 8.  1.  8.]
 [ 7.  0.  0.]
 [14.  8. 13.]
 [ 3.  1.  3.]
 [ 7.  0.  1.]
 [15.  1.  3.]
 [ 2.  1.  1.]
 [ 7.  0.  0.]
 [12.  2. 11.]
 [ 1.  0.  0.]
 [18.  5. 11.]
 [ 7.  3.  6.]
 [ 6.  2.  2.]
 [ 4.  3.  4.]
 [ 3.  0.  2.]]
Current hour only:
Aggregated data, Fraction agreement between EC and EOD: 0.297
Mean of Fraction agreement across indidivuals: 0.292
Standard deviation of Fraction agreement across indidivuals: 0.301

Plus-minus one hour:
Aggregated data, Fraction agreement between EC and EOD: 0.534
Mean of Fraction agreement across indidivuals: 0.554
Standard deviation of Fraction agreement across indidivuals: 0.388


In [34]:
# Compute an anova decomposition using the bernoulli likelihood
# This will test if there are significant differences across
# individuals.

llik_onehour = 0; llik_twohour = 0
for i in range(0, fraction_per_id_onehour.size):
    num_ones_onehour = matching_counts[i,1]
    num_zeros_onehour = matching_counts[i,0] - matching_counts[i,1]
    if num_ones_onehour > 0.0:
        llik_onehour += np.multiply(num_ones_onehour, np.log(fraction_per_id_onehour[i]))
    if num_zeros_onehour > 0.0:
        llik_onehour += np.multiply(num_zeros_onehour, np.log(1-fraction_per_id_onehour[i]))
    num_ones_twohour = matching_counts[i,2]
    num_zeros_twohour = matching_counts[i,0] - matching_counts[i,2]
    if num_ones_twohour > 0.0:
        llik_twohour += np.multiply(num_ones_twohour, np.log(fraction_per_id_twohour[i]))
    if num_zeros_twohour > 0.0:
        llik_twohour += np.multiply(num_zeros_twohour, np.log(1-fraction_per_id_twohour[i]))

agg_num_ones = aggregate_matching_counts[1]
agg_num_zeros = aggregate_matching_counts[0] - aggregate_matching_counts[1]
agg_llik_onehour = agg_num_ones*np.log(aggregate_frac_onehour)+agg_num_zeros*np.log(1-aggregate_frac_onehour)

D_onehour = -2*agg_llik_onehour + 2*llik_onehour

agg_num_ones_twohour = aggregate_matching_counts[2]
agg_num_zeros_twohour = aggregate_matching_counts[0] - aggregate_matching_counts[2]
agg_llik_twohour = agg_num_ones_twohour*np.log(aggregate_frac_twohour)+agg_num_zeros_twohour*np.log(1-aggregate_frac_twohour)

D_twohour = -2*agg_llik_twohour + 2*llik_twohour

from scipy.stats import chi2
n = aggregate_matching_counts[0]
k = matching_counts.shape[0]
df = k-1

print 'ANOVA p-value for current hour: %s' % (1-chi2.cdf(D_onehour, df))
print 'ANOVA p-value for plus-minus one hour: %s' % (1-chi2.cdf(D_twohour, df))


ANOVA p-value for current hour: 1.3191822412217391e-08
ANOVA p-value for plus-minus one hour: 1.1102230246251565e-16


In [38]:
def episode_vs_eod(puff_episode, eod_ema):
    '''
    checks how reliable is the eod_ema in tracking Hand-to-Mouth gestures
    '''
    
    days_smoked = {}
    
    # store smoking events recorded in eod_ema
    for index, row in eod_ema.iterrows():
        if row['status'] == "MISSED":
            continue
        for i in keys:
            if row[i] == 1:
                try:
                    time = datetime.datetime.strptime(str(row['date']), '%m/%d/%Y %H:%M')
                except:
                    time = datetime.datetime.strptime(str(row['date']), '%Y-%m-%d %H:%M:%S')
                date = (time.year, time.month, time.day)
                if time.hour  == 0 or time.hour == 1:
                    date = (time.year, time.month, time.day-1)
                if row['participant_id'] not in days_smoked:
                    days_smoked[row['participant_id']] = set()
                days_smoked[row['participant_id']].add(date)
                break

    count = 0 
    missing = 0
    missing_days = 0
    lst = []
    events = set()
    
    # store HTMGs 
    for index, row in puff_episode.iterrows():
        try:
            count += 1
            try:
                time = datetime.datetime.strptime(str(row['date']), '%m/%d/%y %H:%M')
            except:
                time = datetime.datetime.strptime(str(row['date']), '%Y-%m-%d %H:%M:%S')
            date = (time.year, time.month, time.day)
            if row['participant_id'] not in days_smoked:
                continue
            if date not in days_smoked[row['participant_id']]:
                missing += 1
                if date not in events:
                    missing_days += 1
                    events.add(date)
                lst.append(index)
        except:
            continue

    missing_in_eod = puff_episode.ix[lst]
    # percentage of HTMGs missed by eod_ema
    print("Puff-Episode vs. EOD EMA inconsistency percentage by entries is: ", missing/float(count))
    print("Puff-Episode vs. EOD EMA inconsistency percentage by day is: ", missing_days/float(count))
    return None

In [39]:
episode_vs_eod(puff_episode, eod_ema)

('Puff-Episode vs. EOD EMA inconsistency percentage by entries is: ', 0.2981132075471698)
('Puff-Episode vs. EOD EMA inconsistency percentage by day is: ', 0.12075471698113208)


/Library/Python/2.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [40]:
episode_vs_eod(puff_episode_alt, eod_ema_alternative)

('Puff-Episode vs. EOD EMA inconsistency percentage by entries is: ', 0.32727272727272727)
('Puff-Episode vs. EOD EMA inconsistency percentage by day is: ', 0.12727272727272726)


/Library/Python/2.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [41]:
episode_vs_eod(puff_episode_backup, eod_ema_backup)

('Puff-Episode vs. EOD EMA inconsistency percentage by entries is: ', 0.2939068100358423)
('Puff-Episode vs. EOD EMA inconsistency percentage by day is: ', 0.12903225806451613)


/Library/Python/2.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
